In [0]:
%run "../includes/configuration"

In [0]:
%run
"../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-23")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
#movies_df = spark.sql("select * from movie_silver.movies").filter(f"file_date = '{v_file_date}'")
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                    .filter(f"file_date = '{v_file_date}'")


In [0]:
countries_df =  spark.read.format("delta").load(f"{silver_folder_path}/countries") 
                   


In [0]:
production_country_df =  spark.read.format("delta").load(f"{silver_folder_path}/production_country") \
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
#movie_company_df = spark.sql("select * from movie_silver.movie_company").filter(f"file_date = '{v_file_date}'")
movie_company_df = spark.read.format("delta").load(f"{silver_folder_path}/movie_company") \
                         .filter(f"file_date = '{v_file_date}'")

In [0]:
#production_company_df = spark.sql("select * from movie_silver.production_company").filter(f"file_date = '{v_file_date}'")
production_company_df = spark.read.format("delta").load(f"{silver_folder_path}/production_company") \
                         .filter(f"file_date = '{v_file_date}'")

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
result_df_task = production_country_df.join(movies_df, movies_df.movie_id == production_country_df.movie_id, "inner") \
                    .join(countries_df, countries_df.country_id == production_country_df.country_id, "inner") \
                    .join(movie_company_df, movie_company_df.movie_id == production_country_df.movie_id, "inner") \
                    .join(production_company_df, production_company_df.company_id == movie_company_df.company_id, "inner") \
                    .select(countries_df.country_id,production_company_df.company_id,movies_df.movie_id, movies_df.title,movies_df.budget,movies_df.revenue,movies_df.duration_time,movies_df.release_date,countries_df.country_name,
                            production_company_df.company_name) \
                    .filter(movies_df.year_release_date >= "2010") \
                    .withColumn("created_date", lit(v_file_date)) \
                    .orderBy(movies_df.title.asc())


                    
                    

#### Escribir datos en datalake en formato "Parquet"

In [0]:
#overwrite_partition(result_df_task, "movie_gold", "results_country_prod_company", "created_date") 

In [0]:
#result_df_task.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_country_prod_company")

#### Escribir datos en datalake en formato Delta

In [0]:
merge_key = 'tgt.country_id = src.country_id and tgt.movie_id = src.movie_id and tgt.company_id = src.company_id'
merge_delta_data(result_df_task,"/mnt/moviehistory7809/gold","movie_gold", "results_country_prod_company", "created_date", merge_key)


In [0]:
%sql
select created_date,count(*) 
from movie_gold.results_country_prod_company
group by created_date